In [1]:
import torch

In [2]:
from torch import nn

In [3]:
from torch.utils.data import DataLoader

In [4]:
from torchvision import datasets

In [5]:
from torchvision.transforms import ToTensor

In [6]:
training_data = datasets.FashionMNIST(
    root = "data", train = True, download =True,transform = ToTensor(),
)

test_data = datasets.FashionMNIST(
    root = "data", train = False,download=True,transform = ToTensor(),
)

In [7]:
batch_size = 64
print(len(training_data))
print(len(test_data))
train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

for X, y in test_dataloader:
    print(f"Shape of X [N,C,H,W]:{X.shape}" )
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

60000
10000
Shape of X [N,C,H,W]:torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [9]:
#  모델 정의
class NerualNetwork(nn.Module):
    def __init__(self):
        super(NerualNetwork,self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10)
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logit = self.linear_relu_stack(x)
        return logit
    
model = NerualNetwork().to(device)
print(model)

NerualNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [10]:
# loss function 정의
loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [11]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X,y) in enumerate(dataloader):
        X,y = X.to(device), y.to(device)
        
        pred = model(X)
        loss= loss_fn(pred, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f'Loss: {loss:7f}[{current:>5d}/{size:>5d}]')

In [12]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    # 꼭 넣어야한다.
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= num_batches
    correct /= size
    print(f"test Error : \n Accruacy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}\n")

In [13]:
epochs =5 
for t in range(epochs):
    print(f"Epoch {t+1} \n=============================")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model,loss_fn)
    optimizer.step()
    
print("Done!")

Epoch 1 
Loss: 2.303512[    0/60000]
Loss: 2.293226[ 6400/60000]
Loss: 2.286226[12800/60000]
Loss: 2.255653[19200/60000]
Loss: 2.253332[25600/60000]
Loss: 2.234198[32000/60000]
Loss: 2.213331[38400/60000]
Loss: 2.200733[44800/60000]
Loss: 2.189119[51200/60000]
Loss: 2.166766[57600/60000]
test Error : 
 Accruacy: 41.1%, Avg loss: 2.162140

Epoch 2 
Loss: 2.141944[    0/60000]
Loss: 2.122997[ 6400/60000]
Loss: 2.135023[12800/60000]
Loss: 2.082312[19200/60000]
Loss: 2.071635[25600/60000]
Loss: 2.038955[32000/60000]
Loss: 2.042037[38400/60000]
Loss: 2.024802[44800/60000]
Loss: 1.977182[51200/60000]
Loss: 1.906080[57600/60000]
test Error : 
 Accruacy: 55.2%, Avg loss: 1.895675

Epoch 3 
Loss: 1.894429[    0/60000]
Loss: 1.891061[ 6400/60000]
Loss: 1.761491[12800/60000]
Loss: 1.791420[19200/60000]
Loss: 1.757054[25600/60000]
Loss: 1.629538[32000/60000]
Loss: 1.685951[38400/60000]
Loss: 1.640075[44800/60000]
Loss: 1.587102[51200/60000]
Loss: 1.521678[57600/60000]
test Error : 
 Accruacy: 61.2